In [1]:
import numpy as np
import plotly
import plotly.graph_objs as go
import torch
from torch.utils.data import DataLoader
from experiment import Net, InitDataset, CreateModel, LoadCheckpoint
from yaml import safe_load
import torch.nn as nn

exphash = '3539feb62'

exp = 'results/%s' % exphash
with open(exp) as f:
  h = safe_load(f.read())
h = h['exp_info']  

ds = InitDataset(h)


model = CreateModel(h['hyperparameters'])
LoadCheckpoint(model, 'models/%s'%exphash)

model = model.cuda()


[16:47:29] Loaded model: <All keys matched successfully>


In [20]:
ds.embedding_noise=.2

In [21]:
train_loader = DataLoader(ds, batch_size=512, shuffle=False)


In [22]:
ds[0]

{'char_labels': tensor([34, 55, 40, 43, 36,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        device='cuda:0'),
 'tok_labels': tensor(0, device='cuda:0'),
 'mask': tensor([ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False],
        device='cuda:0'),
 'features': tensor([-1.8841e-02, -5.8079e-02, -1.7125e-02, -4.3478e-02, -5.6779e-03,
         -5.5306e-03, -6.3217e-02, -2.2997e-02, -8.4180e-04, -6.1126e-02,
         -3.0959e-02, -3.2039e-02, -3.7349e-02, -6.8760e-04, -2.5457e-02,
         -7.0537e-02, -4.9270e-02, -5.5776e-02, -2.2890e-02, -7.3702e-02,
         -2.9980e-02, -3.2732e-02, -1.5479e-02, -2.9116e-02, -1.0089e-01,
         -4.1063e-02, -3.3041e-03, -6.4906e-02,  6.4909e-03, -2.1494e-02,
         -3.9993e-02, -5.7379e-03, -1.6629e-02, -3.3112e-02, -5.6672e-02,
         -5.5726e-02, -3.8217e-02, -3.0221e-02, -6.2386e-02, -3.1018e-02,
         -4.9745e-02, -2.4441e-02, -6.5926e-02, -3.

In [23]:
ds[3000:3010]

{'char_labels': tensor([[61, 48, 58, 48, 59, 44, 43,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [52, 40, 59, 59,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [40, 42, 47, 48, 44, 61, 44, 43,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [43, 44, 45, 44, 53, 42, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [48, 53, 59, 44, 57, 53, 40, 51,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [55, 44, 57, 58, 48, 40, 53,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [52, 44, 59, 47, 54, 43, 58,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [46, 48, 53, 46,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [40, 57, 57, 44, 58, 59, 44, 43,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [54, 59, 47, 44, 57, 62, 48, 58, 44,  0,  0,  0,  0,  0,  0,  0,  0,  0]],
        device='cuda:0'),
 'tok_labels': tensor([3000, 3001, 3002, 3003, 3004, 3005, 3006, 3007, 3008, 3009],
        device='cuda:0'),
 'mask': tensor([[

In [24]:
xent = nn.CrossEntropyLoss()
def xent_loss_fn(output, labels):
  return xent(output.view(-1, output.size(-1)), labels.view(-1))

# returns char acc and word acc
def acc_fn(output, labels):
  top = output.argmax(-1)
  right = top==labels
  char_acc = right.float()
  word_acc = (right.sum(-1)==right.shape[-1]).float()
  return char_acc, word_acc


In [25]:
Torch2Py = lambda x: x.cpu().numpy().tolist()

In [26]:
def Validate(val_loader, model):
  cum_char_acc = 0
  cum_word_acc = 0
  cum_xent_loss = 0

  right_chars = []
  right_words = []
  losses_ = []
  maxes = []
  labels_ = []

  for i, data in enumerate(val_loader):
    inputs = data['features']
    labels = data['char_labels']
    labels_ += Torch2Py(labels)
    with torch.no_grad():
      outputs = model(inputs)
      maxes += Torch2Py(outputs.argmax(-1))
      xent_loss = xent_loss_fn(outputs, labels)
      cum_xent_loss += xent_loss

      char_acc, word_acc = acc_fn(outputs, labels)
      right_chars += Torch2Py(char_acc)
      right_words += Torch2Py(word_acc)
      cum_char_acc += char_acc.mean()
      cum_word_acc += word_acc.mean()

  steps = i+1
  print(cum_xent_loss/steps, cum_char_acc/steps, cum_word_acc/steps)
  return right_chars, right_words, maxes, labels_


In [27]:
right_chars, right_words, maxes, labels = Validate(train_loader, model)

tensor(0.0135, device='cuda:0') tensor(0.9955, device='cuda:0') tensor(0.9609, device='cuda:0')


In [28]:
len(right_chars), len(right_words), len(maxes), len(labels),

(25082, 25082, 25082, 25082)

In [29]:
idx_to_char_map= {v:k for k,v in ds.char_to_idx_map.items()}

In [30]:
def GetChars(encoded):
  s = ""
  for c in encoded:
    s+= idx_to_char_map[c]
  return s.replace('\x00','_')

In [31]:
for i, r in enumerate(right_words):
  if r==0 and i%1==0:
    print(GetChars(maxes[i]), GetChars(labels[i]))

[pad]_____]_______ [pad]_____________
[unused0__________ [unused0]_________
[unused6]]________ [unused1]_________
[unused427]_______ [unused2]_________
[unused4]5]_______ [unused3]_________
[unused7]_________ [unused4]_________
[unused4]_]_______ [unused5]_________
[unused1]_________ [unused6]_________
[unused4]]]_______ [unused7]_________
[unused434]_______ [unused8]_________
[unused309]_______ [unused10]________
[unused31]]_______ [unused11]________
[unused9]]________ [unused12]________
[unused33]]_______ [unused13]________
[unused437]_______ [unused14]________
[unused112]_______ [unused15]________
[unused719]_______ [unused16]________
[unused169]_______ [unused17]________
[unused846]_______ [unused18]________
[unused330]_______ [unused19]________
[unused788]_______ [unused20]________
[unused118]_______ [unused21]________
[unused215]_______ [unused22]________
[unused33]]_______ [unused23]________
[unused26]]_______ [unused24]________
[unused236]_______ [unused26]________
[unused770]_

In [225]:
embeddings = ds.data['embeddings']

In [226]:
embeddings.norm(2,-1).mean()

tensor(1.6638, device='cuda:0')

In [252]:
embedding_noise = .25

In [253]:
r = torch.normal(mean=embedding_noise, std=1, size=embeddings.size()).cuda()
norm_factor = torch.norm(r, 2, -1).mean()/embedding_noise
normalized_noise = r/norm_factor
new = embeddings + normalized_noise

In [254]:
((new-embeddings)**2).mean()

tensor(8.1433e-05, device='cuda:0')

In [255]:
torch.sqrt((new-embeddings)**2).mean()

tensor(0.0072, device='cuda:0')

In [217]:
n = new.norm(2,-1)

In [218]:
def MakeHist(h, name=None):
  y = h[0]
  x= h[1]
  return {
    'name': name,
    'type': 'bar',                                                                                                                   'x': x,
          'marker': dict(opacity=.7),
   'y': y
  }


dat = MakeHist(np.histogram(n, bins=30), name='toks')



fig = fig = go.Figure(data=[dat], layout={'barmode':'overlay'})
plotly.offline.plot(fig, filename='/home/tucker/Downloads/imdb_size_distci.html')


'/home/tucker/Downloads/imdb_size_distci.html'

tensor([0.0041, 0.0040, 0.0039,  ..., 0.0039, 0.0041, 0.0039])